In [1]:
import spacy
import pandas as pd
from spacy import displacy
import os
import re

In [2]:
# Cargar el archivo CSV
df = pd.read_csv(r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\limpios\dialogos_procesados.csv")
print(df.head())

                  titulo                                     contenido_dial  \
0               Clitofon   Sócrates Clitofon, hijo de Aristonimo, me han...   
1        Timeo de Locres   He aquí lo que Timeo de Locres dice: Hay dos ...   
2  El segundo Alcibíades   Sócrates Alcibíades, ¿vas a orar en este temp...   
3                  Fedón   Equecrates Fedón, ¿estuviste tú mismo cerca d...   
4            El banquete   Apolodoro Me considero bastante preparado par...   

                                    contenido_limpio  
0  Sócrates Clitofon, hijo de Aristonimo, me han ...  
1  He aquí lo que Timeo de Locres dice: Hay dos c...  
2  Sócrates Alcibíades, ¿vas a orar en este templ...  
3  Equecrates Fedón, ¿estuviste tú mismo cerca de...  
4  Apolodoro Me considero bastante preparado para...  


In [3]:
# Asignar ID_obra empezando desde 1
df['ID_obra'] = range(1, len(df) + 1)

# Reordenar las columnas
df = df[['ID_obra', 'titulo', 'contenido_dial', 'contenido_limpio']]

print(df)

    ID_obra                 titulo  \
0         1               Clitofon   
1         2        Timeo de Locres   
2         3  El segundo Alcibíades   
3         4                  Fedón   
4         5            El banquete   
5         6                  Lisis   
6         7               Eutidemo   
7         8                    Ion   
8         9                  Fedro   
9        10                 Filebo   
10       11                 Laques   
11       12               Carmides   
12       13      Critón o el deber   
13       14               Eutifrón   
14       15   El primer Alcibiades   
15       16   Apología de Sócrates   

                                       contenido_dial  \
0    Sócrates Clitofon, hijo de Aristonimo, me han...   
1    He aquí lo que Timeo de Locres dice: Hay dos ...   
2    Sócrates Alcibíades, ¿vas a orar en este temp...   
3    Equecrates Fedón, ¿estuviste tú mismo cerca d...   
4    Apolodoro Me considero bastante preparado par...   
5    Sócrat

In [4]:
# Carga modelo español large
nlp = spacy.load('es_core_news_lg')

c:\Users\Lucas\AppData\Local\Programs\Python\Python311\Lib\site-packages\spacy\util.py:910: UserWarning: [W095] Model 'es_core_news_lg' (3.5.0) was trained with spaCy v3.5.0 and may not be 100% compatible with the current version (3.8.4). If you see errors or degraded performance, download a newer compatible model or retrain your custom model with the current spaCy version. For more details and available updates, run: python -m spacy validate
  warnings.warn(warn_msg)


In [5]:
# Función para segmentar por líneas de diálogo, asignando el interlocutor a cada línea
def segmentar_lineas_con_interlocutor(texto):
    pattern = r"^(?P<interlocutor>[A-ZÁÉÍÓÚÑ][a-záéíóúñ]+):\s*(?P<contenido>.+)$"
    lineas = texto.split("\n")
    datos = []
    for linea in lineas:
        match = re.match(pattern, linea.strip())
        if match:
            datos.append({
                "interlocutor": match.group("interlocutor"),
                "contenido": match.group("contenido").strip()
            })
    return datos

In [6]:
# Aplicar la segmentación de líneas por obra
df["dialogos_segmentados"] = df["contenido_limpio"].apply(segmentar_lineas_con_interlocutor)


In [7]:
# Función para extraer preguntas socráticas (de Sócrates) dentro de un diálogo
def extraer_preguntas(texto):
    doc = nlp(texto)
    preguntas = [sent.text for sent in doc.sents if "?" in sent.text]
    return preguntas

In [8]:
# Aplicar extracción de preguntas
df["preguntas_socraticas"] = df["contenido_limpio"].apply(extraer_preguntas)

In [9]:
for index, fila in df.iterrows():
    preguntas = fila["preguntas_socraticas"]
    titulo = fila["titulo"]
    
    if preguntas:
        print(f"Fila {index} - Título {titulo}:")
        for pregunta in preguntas:
            print(f"  - {pregunta}")
        print("-" * 40)

Fila 0 - Título Clitofon:
  - Me ha sucedido muchas veces, Sócrates, que encontrándome contigo, me he dejado llevar de la más viva admiración al oír tus discursos, y me ha parecido que hablabas mejor que nadie, cuando reprendiendo a los hombres, como un dios que aparece en lo alto de una máquina de teatro «¿A dónde vais a parar, mortales?
  - ¿No veis que no hacéis nada de lo que deberíais practicar?
  - Cuando vosotros y vuestros hijos, después de conocer las letras, la música y la gimnástica, lo cual creéis que constituye la educación más perfecta, veis que no sois menos ignorantes por lo que hace al uso que hacéis de vuestras riquezas, ¿cómo no os escandalizáis de esta educación, y no buscáis maestros que hagan desaparecer esta ignorancia y esta disonancia?
  - ¿Qué hombre seria capaz de escoger voluntariamente un mal semejante?
  - Pero si la victoria depende de la voluntad, ¿la derrota no es siempre involuntaria?
  - En primer término me dirigí a los que tú más estimas, preguntánd

In [12]:
# Función para extraer el diálogo socrático
def extraer_dialogo_socratico(texto):
    lineas = texto.split("\n")
    dialogo_socratico = [linea.strip() for linea in lineas if linea.strip().startswith("Sócrates")]
    return dialogo_socratico  # Devolvemos una lista de frases, no un string

In [13]:
df["dialogo_socratico"] = df["contenido_limpio"].apply(extraer_dialogo_socratico)

In [14]:
for index, fila in df.iterrows():
    dialogos = fila["dialogo_socratico"]
    titulo = fila["titulo"]
    
    if dialogos:
        print(f"\n📘 Obra: {titulo} (Fila {index})")
        print("🗣️ Diálogo de Sócrates:")
        for dialogo in dialogos:
            print(f"  - {dialogo}")
        print("-" * 60)



📘 Obra: Clitofon (Fila 0)
🗣️ Diálogo de Sócrates:
  - Sócrates Clitofon, hijo de Aristonimo, me han dicho hace un instante, que en una conversación que has tenido con Licias, has criticado las discusiones filosóficas de Sócrates, y puesto en las nubes las lecciones de Trasimaco Clitofon Te han referido exactamente, Sócrates, lo que he dicho de ti a Licias; si en unas cosas te he censurado, también te he alabado en otras, y como veo en claro, que a pesar de tu aire de indiferencia estás incomodado conmigo, seria conveniente, ya que estamos solos, repetirte lo mismo que he dicho, y te desengañarás de que no soy injusto para contigo. Indudablemente te han informado mal, y esta es la causa de tu irritación. Pero si me permites decirte todo lo que pienso, estoy pronto a hacerlo, y no te ocultaré nada. Sócrates No tendría razón para oponerme a tu deseo, cuando éste redunda en mi provecho, porque evidentemente desde el momento que me hagas ver el bien y el mal que  residen en mí, procuraré s

In [15]:
# Función para extraer preguntas y respuestas básicas
def preguntas_respuestas_por_frase(texto):
    doc = nlp(texto)
    resultados = []
    ultima_pregunta = None

    for sent in doc.sents:
        frase = sent.text.strip()
        
        if frase.startswith("Sócrates") and frase.endswith("?"):
            ultima_pregunta = frase
            continue

        if ultima_pregunta and not frase.startswith("Sócrates") and len(frase) > 5:
            match = re.match(r"^([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+)", frase)
            orador = match.group(1) if match else "Desconocido"
            resultados.append({
                "pregunta": ultima_pregunta,
                "respuesta": frase,
                "respondedor": orador
            })
            ultima_pregunta = None

    return resultados


In [16]:
df["preguntas_respuestas_basicas"] = df["contenido_limpio"].apply(preguntas_respuestas_por_frase)

In [17]:
for _, fila in df.iterrows():
    titulo = fila["titulo"]
    pares = fila["preguntas_respuestas_basicas"]

    if pares:
        print(f"\n🎭 Obra: {titulo}")
        for par in pares:
            print(f"❓ Sócrates: {par['pregunta']}")
            print(f"💬 {par['respondedor']}: {par['respuesta']}")
        print("-" * 60)


🎭 Obra: El segundo Alcibíades
❓ Sócrates: Sócrates Alcibíades, ¿vas a orar en este templo?
💬 Alcibíades: Alcibíades Sí, Sócrates.
❓ Sócrates: Sócrates A mí me parece que hay materia para pensar seriamente, porque, ¡en nombre de Júpiter!, ¿no crees que entre las cosas que pedimos a los dioses, sea en público, sea en secreto, hay unas que se nos conceden y otras que se nos niegan, y que tan pronto atienden como desechan nuestras súplicas?
💬 Alcibíades: Alcibíades Sí lo creo.
❓ Sócrates: Sócrates Y bien, ¿no te parece que la oración exige mucha  prudencia, porque sin saberlo, pueden pedirse a los dioses grandes males, creyendo pedirles bienes, y los dioses no encontrarse en disposición de conceder lo que se les pide?
💬 Por: Por ejemplo, Edipo les pidió en un arrebato de cólera, que sus hijos decidiesen con la espada sus derechos hereditarios, y cuando debía pedir a los dioses que le libraran de las desgracias de que era víctima, atrajo sobre sí otras nuevas; porque fueron escuchados sus 

In [18]:
# Función para extraer preguntas implícitas
def extraer_preguntas_implicitas(texto):
    doc = nlp(texto)
    preguntas = []

    patrones_pregunta = [
        r"^(Dime|Explícame|Hazme saber|Enséñame|Muéstrame|Cuéntame)\b",
        r"^Entonces.*\b(qué|cómo|por qué|cuál)\b",
        r"^¿?Qué crees.*",  # sin ? inicial
    ]

    for sent in doc.sents:
        frase = sent.text.strip()
        if "?" in frase:
            preguntas.append(frase)
        else:
            for patron in patrones_pregunta:
                if re.match(patron, frase, re.IGNORECASE):
                    preguntas.append(frase)
                    break

    return preguntas

In [19]:
df["preguntas_implicitas"] = df["contenido_limpio"].apply(extraer_preguntas_implicitas)

In [16]:
for _, fila in df.iterrows():
    titulo = fila["titulo"]
    preguntas = fila["preguntas_implicitas"]

    if preguntas:
        print(f"\n📘 Obra: {titulo}")
        for pregunta in preguntas:
            print(f"❓ {pregunta}")


📘 Obra: Clitofon
❓ Me ha sucedido muchas veces, Sócrates, que encontrándome contigo, me he dejado llevar de la más viva admiración al oír tus discursos, y me ha parecido que hablabas mejor que nadie, cuando reprendiendo a los hombres, como un dios que aparece en lo alto de una máquina de teatro «¿A dónde vais a parar, mortales?
❓ ¿No veis que no hacéis nada de lo que deberíais practicar?
❓ Cuando vosotros y vuestros hijos, después de conocer las letras, la música y la gimnástica, lo cual creéis que constituye la educación más perfecta, veis que no sois menos ignorantes por lo que hace al uso que hacéis de vuestras riquezas, ¿cómo no os escandalizáis de esta educación, y no buscáis maestros que hagan desaparecer esta ignorancia y esta disonancia?
❓ ¿Qué hombre seria capaz de escoger voluntariamente un mal semejante?
❓ Pero si la victoria depende de la voluntad, ¿la derrota no es siempre involuntaria?
❓ En primer término me dirigí a los que tú más estimas, preguntándoles qué objeto debe

In [20]:
# Función para extraer preguntas y respuestas implícitas
def extraer_preguntas_respuestas_implicitas(texto):
    doc = nlp(texto)
    resultados = []
    ultima_pregunta = None

    patrones_pregunta = [
        r"^(Dime|Explícame|Hazme saber|Enséñame|Muéstrame|Cuéntame)\b",
        r"^Entonces.*\b(qué|cómo|por qué|cuál)\b",
        r"^¿?Qué crees.*",
    ]

    for sent in doc.sents:
        frase = sent.text.strip()

        # ¿Es una pregunta explícita o implícita?
        if "?" in frase or any(re.match(p, frase, re.IGNORECASE) for p in patrones_pregunta):
            ultima_pregunta = frase
            continue

        # Si hay una pregunta previa y esta frase podría ser respuesta:
        if ultima_pregunta and not frase.startswith("Sócrates") and len(frase) > 5:
            # Intentamos detectar orador (opcional)
            match = re.match(r"^([A-ZÁÉÍÓÚÑ][a-záéíóúñ]+):?", frase)
            orador = match.group(1) if match else "Desconocido"
            resultados.append({
                "pregunta": ultima_pregunta,
                "respuesta": frase,
                "respondedor": orador
            })
            ultima_pregunta = None

    return resultados

In [21]:
df["preguntas_respuestas_implicitas"] = df["contenido_limpio"].apply(extraer_preguntas_respuestas_implicitas)


In [19]:
for _, fila in df.iterrows():
    titulo = fila["titulo"]
    pares = fila["preguntas_respuestas_implicitas"]

    if pares:
        print(f"\n🎭 Obra: {titulo}")
        for par in pares:
            print(f"❓ Sócrates: {par['pregunta']}")
            print(f"💬 {par['respondedor']}: {par['respuesta']}")
        print("-" * 60)



🎭 Obra: Clitofon
❓ Sócrates: ¿No veis que no hacéis nada de lo que deberíais practicar?
💬 El: El objeto de todos vuestros cuidados es amontonar riquezas y trasmitirlas a vuestros hijos, sin inquietaros para nada del uso que puedan hacer de ellas.
❓ Sócrates: Cuando vosotros y vuestros hijos, después de conocer las letras, la música y la gimnástica, lo cual creéis que constituye la educación más perfecta, veis que no sois menos ignorantes por lo que hace al uso que hacéis de vuestras riquezas, ¿cómo no os escandalizáis de esta educación, y no buscáis maestros que hagan desaparecer esta ignorancia y esta disonancia?
💬 Desconocido: A causa de este desorden y de esta inconveniencia, y no porque un pié deje de guardar compás con la lira, tiene lugar la falta de acuerdo y armonía entre hermanos y hermanos, entre Estados y Estados, y en sus divisiones y en sus guerras sufren el cúmulo de males que mutuamente se causan.
❓ Sócrates: ¿Qué hombre seria capaz de escoger voluntariamente un mal sem

In [20]:
# Función para extraer entidades nombradas de los diálogos socráticos
def extraer_entidades(texto):
    doc = nlp(texto)
    entidades = [(ent.text, ent.label_) for ent in doc.ents]
    return entidades

In [21]:
# Aplicar extracción de entidades
df["entidades"] = df["contenido_limpio"].apply(extraer_entidades)

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16 entries, 0 to 15
Data columns (total 10 columns):
 #   Column                           Non-Null Count  Dtype 
---  ------                           --------------  ----- 
 0   ID_obra                          16 non-null     int64 
 1   titulo                           16 non-null     object
 2   contenido_dial                   16 non-null     object
 3   contenido_limpio                 16 non-null     object
 4   preguntas_socraticas             16 non-null     object
 5   dialogo_socratico                16 non-null     object
 6   preguntas_respuestas_basicas     16 non-null     object
 7   preguntas_implicitas             16 non-null     object
 8   preguntas_respuestas_implicitas  16 non-null     object
 9   entidades                        16 non-null     object
dtypes: int64(1), object(9)
memory usage: 1.4+ KB


In [23]:
import json
import os

# Extraer columnas necesarias
datos_exportar = df[["ID_obra", "titulo", "preguntas_socraticas", "dialogo_socratico", "preguntas_respuestas_basicas", "preguntas_implicitas", "preguntas_respuestas_implicitas"]]

# Convertir a lista de dicts (formatio JSNON nativo)
datos_json =datos_exportar.to_dict(orient="records")


In [24]:
output_directory =r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed"

output_path = os.path.join(output_directory, "dialogos_nlp.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(datos_json, f, ensure_ascii=False, indent=4)

In [25]:
# Definir el directorio de salida
output_directory = r"C:\Users\Lucas\Desktop\Socrates\llm_proyect\data\procesed"

# Guardar el DataFrame procesado
df.to_csv(os.path.join(output_directory, "dialogos_socraticos_procesados.csv"), index=False)